### ELN WRITEUP COMPARISON BETWEEN DM INSTANCES:

- run each 'cell' one by one; once you get to the 3rd cell:
- simply select the CRO protocol name and click 'Submit' button
- it will read the list of experiment ids from the text file
- afterwards an html render of those specific experiment ids will be executed

In [ ]:
pip install psycopg2-binary

In [ ]:
from os import getenv
from dotenv import load_dotenv
load_dotenv()
DB_USER = getenv("DB_USER")
DB_PASS = getenv("DB_PASS")
DB_HOST = getenv("DB_HOST")
DB_PORT = getenv("DB_PORT")
DB_NAME = getenv("DB_NAME")

In [ ]:
%run compare_eln_writeup_scrape.ipynb

In [ ]:
dropdown = widgets.Dropdown(
    options=cro_prots,
    value=cro_prots[0],
    description="Select:",
    style={"description_width": "initial"},
)

button = widgets.Button(description="Submit")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print(dropdown.value)

button.on_click(on_button_click)
display(dropdown, button, output)

### ONLY RUN BELOW AFTER EXECUTING EACH CELL ABOVE AND SELECTING THE CRO PROTOCOL THEN CLICK 'SUBMIT'

In [ ]:
idx = 0
dd_value = dropdown.value
print ('='*15, f'{dd_value}', '='*15)
html_output = ""
exp_ids = []
with open(os.path.join(dir_path, file_name_prefix.format(dd_value)), 'r') as file:
    for line in file:
        exp_ids.extend(line.strip().split(","))
        
filtered_df = df[df['exp_id'].isin(exp_ids)].copy()
filtered_df['write_up'] = filtered_df.apply(color_code_writeup, axis=1) 

for exp_id, group in filtered_df.groupby('exp_id'):
    idx += 1
    html_output += render_html_group(idx, group)
html_output += js_code
display(HTML(html_output))

### Run the following cell and click the 'Update Validity' blue button

In [ ]:
%%javascript

const div = document.createElement('div');
div.innerHTML = '<p style="margin-top: 10px; font-size: 20px; color: gray;">Click the button only after having finished validating each experiment ID.</p>';
element.append(div);

const updateButton = document.createElement('button');
updateButton.textContent = 'Update validity';
updateButton.style.padding = '12px 24px';
updateButton.style.fontSize = '18px';
updateButton.style.borderRadius = '8px';
updateButton.style.backgroundColor = '#007BFF';
updateButton.style.color = 'white';
updateButton.style.border = 'none';
updateButton.style.cursor = 'pointer';
updateButton.style.marginTop = '10px';
element.append(updateButton);
updateButton.addEventListener('click', function() {
    postValidityBackend();
    div.innerHTML = '<p style="margin-top: 10px; font-size: 20px; color: gray;">Updated validity data (run the following cell to export to file & view)</p>';
});

### Run the following cell to export to the current directory as .csv

In [ ]:
df_val = pd.read_sql(sql_query.format(f"validity_{user_name}") , engine)
df.to_csv(os.path.join(os.path.expanduser("~"), f"{user_name}_exp_id_validation_output.csv"), index=False)
df_val